In [ ]:
# When using a colab notebook:
#!wget https://github.com/Aleph-Alpha/examples/blob/main/bootcamp/data.md
#!wget https://github.com/Aleph-Alpha/examples/blob/main/requirements.txt
#!pip install -r requirements.txt

In [ ]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse, SemanticEmbeddingRequest, SemanticEmbeddingResponse, SemanticRepresentation
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

client = Client(token=os.getenv("AA_TOKEN"))

### Step 1: Let's do a really simple QA prompt
In this step we will ask luminous a question from the manual. Let's see how it answers.
Write a completionrequest and make luminous answer it. Don't forget the stop sequences.
You can find the documentation for the completionrequest here: https://docs.aleph-alpha.com/docs/tasks/complete/

In [ ]:
prompt = "Q: What is stop category 0?\nA:"

# TODO: Write the completion request
request = None 

# TODO: Call the client to run the completionrequest
response = client.complete(request, model="luminous-base-control")
print(response.completions[0].completion)

#### Step 2: Let's do a QA prompt with a context
Obviously, we can't expect luminous to answer the question if we don't give it any context. Let's see how it answers when we give it some context.
Write the new prompt and make luminous answer it. Don't forget the stop sequences.

In [ ]:
context = """The robot has protective and emergency stop functions (stop category 0 or 1, in accordance with IEC 60204-1).

| Stop category 0 | As defined in IEC 60204-1, stopping by immediate removal of power to the machine actuators. |  
|---|---|  
| Stop category 1 | As defined in IEC 60204-1, a controlled stop with power avail- able to the machine actuators to achieve the stop and then re- moval of power when the stop is achieved. |  """

# TODO write a prompt that makes luminous answer the question based on the context
prompt = f"""Prompt"""

request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=32)

response = client.complete(request, model="luminous-base-control")
print(response.completions[0].completion)



#### Step 3: Let's use our search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question
This step is a bit long, but it is a good exercise to see how we can use the search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question.

In [ ]:
# Read the data in the data.md file
with open("new_data.md", "r") as f:
    data = f.read()
    
# Split the data into a list of texts
texts = data.split("#")
texts = texts[1:]

print(f"data: {data[:100]}")


In [ ]:
question = "What are current macro trends in the market?"

#TODO: embed the contexts

embedded_texts = []
for text in texts:
    pass # TODO create the embeddings and add them to the list

# TODO: embed the question
embedded_question = None

# calculate the cosine similarity between the question and the contexts
similarities = []
for embedded_text in embedded_texts:
    similarities.append(1 - spatial.distance.cosine(embedded_text, embedded_question))

# Get the text in the most similar context
selected_context = texts[np.argmax(similarities)]

# TODO: write a prompt that makes luminous answer the question based on the context
prompt = f"""Prompt"""

# TODO: write the CompletionRequest
request = None # TODO: create the request

response = client.complete(request, model="luminous-base-control")

print("Model response: ", response.completions[0].completion)
print("\n\n selected_context: ", selected_context)
print(similarities)

### Let's try this with a more complex setup and a vector database

In [ ]:
# First we spin up the Qdrant server
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, Batch

q_client = QdrantClient(path="db")

q_client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)



In [ ]:
# Let's create embeddings for each of the texts and store them in a list
embeddings = []
for text in texts:
    # TODO: embed the texts
    embeddings.append(client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding)

In [ ]:
# now we can upsert the data into Qdrant
ids = list(range(len(texts)))
payloads = [{"text": text} for text in texts]

q_client.upsert(
     collection_name="test_collection",
     points=Batch(
     ids=ids,
     payloads=payloads,
     vectors=embeddings
     )
)

In [ ]:

# write a function that takes a question and returns an answer by searching in the Qdrant database
def search_and_answer(question):
    # First we embed the question
    embedded_question = client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Query, compress_to_size=128), model="luminous-base").embedding
    
    # Then we search for the most similar text
    search_result = q_client.search(
        collection_name="test_collection",
        query_vector=embedded_question,
        filter=None,
        top=1
    )
    
    # return "no answer found" if no result has a score above 0.3
    if search_result[0].score < 0.3:
        return "no answer found"
    
    # Then we get the text from the search result
    text = search_result[0].payload["text"]
    
    # Finally we ask luminous to answer the question based on the text
    prompt = f"""### Instructions: Answer the question briefly based on the provided Input.
    
    ### Input: {text}
    
    ### Question: {question}
    
    ### Response:"""
    
    # TODO write the CompletionRequest
    request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=64, stop_sequences=["###"])
    
    # TODO get the response from luminous
    response = client.complete(request, model="luminous-base-control")
    
    return response.completions[0].completion

In [ ]:
search_and_answer("What is the capital of France?")

In [ ]:
search_and_answer("What are current macro trends in the market?")